In [22]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
import pandas as pd
# Bibliotecas Auxiliares
import seaborn as sns
import numpy as np
import collections
import matplotlib.pyplot as plt
import nest_asyncio
nest_asyncio.apply()
import tensorflow_federated as tff
np.random.seed(0)
from collections import Counter

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow import keras

import collections
import functools
import os
import time
from scipy import stats

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

import datetime


print(tf.__version__)

/bin/bash: /home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2.10.0


In [4]:
RRR = [1,2]
RRR2 = [3,4]
np.concatenate((RRR, RRR2))

array([1, 2, 3, 4])

In [27]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    array = []
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    results['matrix'] = matrix
    results['TP'] = matrix[0][0]
    results['FP'] = matrix[0][1]
    results['FN'] = matrix[1][0]
    results['TN'] = matrix[1][1]
    
    array.append(accuracy)
    array.append(precision)
    array.append(recall)
    array.append(f1)
    array.append(kappa)
    array.append(auc)
    array.append(matrix)
    array.append(matrix[0][0]) # TP
    array.append(matrix[0][1]) # FP
    array.append(matrix[1][0]) # FN
    array.append(matrix[1][1]) # TN
    
    return results, array

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
    return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score]
    
def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))

In [6]:
# configs
EPOCHS = 3
NUM_EPOCHS = EPOCHS
BATCH_SIZE = 32

TIME_SERIES_SIZE = 4   # Determines the window size. Ex (4,9)
TIME_STEP_SHIFT  = 1   # Determines specifies the number of steps to move the window forward at each iteration.

baseFolder = "../data_2019_processed/"

# selected features
inputFeatures = ["activity","location","day_of_week",
                 "light","phone_lock","proximity",
                 "sound","time_to_next_alarm", "minutes_day"]
# outputs
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]

In [7]:
# test data comprising 25% of the data. It must be fixed to all models being evaluated
X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")

# input folder
inputFolders = baseFolder
           
dsTrain = ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
            '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
            '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
            '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
            'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
            'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
            'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
            'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
            'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
            'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', 
            'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
            'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
            'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
            'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
            'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
            'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
            'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
            'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', 
            'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
            'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
            'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
            'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4', 
            'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
            'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
            'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
            'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
            'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
            'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']


# client datasets used on the training process
dsTrain =  ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
            '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
            '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
            '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
            'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
            'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
            'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
            'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
            'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
            'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', 
            'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
            'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
            'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
            'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
            'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
            'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
            'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
            #['PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc'], 
            'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
            #['rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw'], 
            #['RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI'], 
            'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'] 
            #['VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is'], 
            #['Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw'], 
            #['XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA'], 
            #['YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw'], 
            #['ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM'], 
            #['ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']]

In [8]:
# load cliend data
clientList = []

# balancing classes
#oversample = RandomOverSampler(sampling_strategy='auto') #minority
#undersample = RandomUnderSampler(sampling_strategy='majority')

for i in range(0,len(dsTrain)):
    print (i," ", str(inputFolders)+"student_"+dsTrain[i]+"_numeric.csv") #_numeric
    # load client data
    dataset = pd.read_csv(inputFolders+"student_"+dsTrain[i]+"_numeric.csv")
    
    # print(dataset)
    y_train = dataset['class'].copy()
    
    # does not add datasets that dont have instances from both classes
    if y_train.sum() != 0 and (y_train.sum() != len(y_train)):
        X_train = dataset
        # balancing classes
        #X_train, y_train = oversample.fit_resample(dataset, y_train)
        #X_train, y_train = undersample.fit_resample(dataset, y_train)
        
        X_train['class'] = y_train
        
        clientList.append(X_train)
        
print("Total",(len(clientList)))

0   ../data_2019_processed/student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs_numeric.csv
1   ../data_2019_processed/student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA_numeric.csv
2   ../data_2019_processed/student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0_numeric.csv
3   ../data_2019_processed/student_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys_numeric.csv
4   ../data_2019_processed/student_7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA_numeric.csv
5   ../data_2019_processed/student_a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4_numeric.csv
6   ../data_2019_processed/student_ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc_numeric.csv
7   ../data_2019_processed/student_Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U_numeric.csv
8   ../data_2019_processed/student_CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA_numeric.csv
9   ../data_2019_processed/student_DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc_numeric.csv
10   ../data_2019_processed/student_DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA_numeric.csv
11   ../

In [9]:
# undestand the dataset by looking on their infos
print(X_test.info())

X_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float64
 1   location            136286 non-null  float64
 2   timestamp           136286 non-null  float64
 3   day_of_week         136286 non-null  float64
 4   light               136286 non-null  float64
 5   phone_lock          136286 non-null  float64
 6   proximity           136286 non-null  float64
 7   sound               136286 non-null  float64
 8   time_to_next_alarm  136286 non-null  float64
 9   minutes_day         136286 non-null  float64
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  object 
dtypes: float64(10), object(2)
memory usage: 12.5+ MB
None


,activity,location,timestamp,day_of_week,light,phone_lock,proximity,sound,time_to_next_alarm,minutes_day,timestamp_text,class
0,0.00,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.655565,0.906151,0.115358,2018-05-15 02:46:57,asleep
1,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.567296,0.906052,0.116053,2018-05-15 02:47:27,asleep
2,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.567296,0.906052,0.116053,2018-05-15 02:47:57,asleep
3,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.660604,0.905952,0.116748,2018-05-15 02:48:28,asleep
4,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.660604,0.905952,0.116748,2018-05-15 02:48:57,asleep
...,...,...,...,...,...,...,...,...,...,...,...,...
136281,0.25,1.0,0.007015,0.166667,0.000056,1.0,1.0,0.000000,0.000099,0.510076,2018-06-13 12:14:37,awake
136282,0.25,1.0,0.007015,0.166667,0.000078,1.0,1.0,0.000000,0.000694,0.512856,2018-06-13 12:18:08,awake
136283,0.25,1.0,0.007015,0.166667,0.000078,1.0,1.0,0.000000,0.000595,0.513551,2018-06-13 12:19:08,awake
136284,0.25,1.0,0.007016,0.166667,0.000085,1.0,1.0,0.000000,0.000595,0.513551,2018-06-13 12:19:38,awake


In [10]:
# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset

# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the input dataset
for i in range(0,len(clientList)):
    clientList[i] = transform_output_numerical_class_into_one_hot_encoding(clientList[i])
    #print (clientList[i])

In [11]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float64
 1   location            136286 non-null  float64
 2   timestamp           136286 non-null  float64
 3   day_of_week         136286 non-null  float64
 4   light               136286 non-null  float64
 5   phone_lock          136286 non-null  float64
 6   proximity           136286 non-null  float64
 7   sound               136286 non-null  float64
 8   time_to_next_alarm  136286 non-null  float64
 9   minutes_day         136286 non-null  float64
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  object 
 12  awake               136286 non-null  uint8  
 13  asleep              136286 non-null  uint8  
dtypes: float64(10), object(2), uint8(2)
memory usage: 12.7+ MB


In [12]:
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# transforms the data
X_test = transform_data_type(X_test)

X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float32
 1   location            136286 non-null  float32
 2   timestamp           136286 non-null  float64
 3   day_of_week         136286 non-null  float32
 4   light               136286 non-null  float32
 5   phone_lock          136286 non-null  float32
 6   proximity           136286 non-null  float32
 7   sound               136286 non-null  float32
 8   time_to_next_alarm  136286 non-null  float32
 9   minutes_day         136286 non-null  float32
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  object 
 12  awake               136286 non-null  float32
 13  asleep              136286 non-null  float32
dtypes: float32(11), float64(1), object(2)
memory usage: 8.8+ MB


In [13]:
# selects the data to train and test
X_test_data = X_test[inputFeatures]
y_test_label = X_test[outputClasses]

# transform to time series
X_test_data, y_test_label = create_dataset_time_series_with_one_output(    #timestamp
    X_test_data, 
    y_test_label, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)

# transtorm data to tensor slices
client_test_dataset = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_label))
#client_test_dataset = tf.data.Dataset.from_tensor_slices((X_test_data.values, y_test_label.values))

#client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE, drop_remainder=True)
client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)

print(client_test_dataset.element_spec)
client_test_dataset

2023-03-06 12:11:32.884244: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-06 12:11:32.960830: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-06 12:11:32.961464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-06 12:11:32.964989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

(TensorSpec(shape=(None, 4, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))


<BatchDataset element_spec=(TensorSpec(shape=(None, 4, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [14]:
federated_training_data = []
# transform the data
for i in range(0,len(clientList)):
    # selects the data to train and test
    data   = clientList[i][inputFeatures]
    labels = clientList[i][outputClasses]
    # transform data to float32
    #data = transform_data_type(data)

    # transform to time series
    data, labels = create_dataset_time_series_with_one_output(    #timestamp
        data, 
        labels, 
        TIME_SERIES_SIZE, 
        TIME_STEP_SHIFT
    )

    # transform the data to tensor slices
    client_train_dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    #client_train_dataset = tf.data.Dataset.from_tensor_slices((data.values, labels.values))
    # apply the configs
    client_train_dataset = client_train_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)
    # transform the data to
    federated_training_data.append(client_train_dataset)

In [15]:
def create_keras_model():
    model = keras.Sequential()
    model.add(LSTM(128,input_shape=(4, 9,))) #return_sequences=True, 
    #model.add(LSTM(128,input_shape=(2, 9,)))
    #model.add(keras.layers.Dropout(rate=0.5))
    #model.add(LSTM(64,activation="tanh"))
    model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(2, activation='softmax'))#softmax,sigmoid
    
    return model

In [16]:
keras_model = create_keras_model()
#keras_model.summary()
keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               70656     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 70,914
Trainable params: 70,914
Non-trainable params: 0
_________________________________________________________________


In [17]:
def model_fn():
    # We _must_ create a new model here, and _not_ capture it from an external
    # scope. TFF will call this within different graph contexts.
    #keras_model = create_keras_model()
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
      keras_model,
      input_spec=client_train_dataset.element_spec,
      loss=tf.keras.losses.CategoricalCrossentropy(), #BinaryCrossentropy
      metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision()])

In [18]:
fed_avg_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),#client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [19]:
print(fed_avg_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[9,512],
      float32[128,512],
      float32[512],
      float32[128,2],
      float32[2]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64
  >
>@SERVER)


In [20]:
state = fed_avg_process.initialize()

In [23]:
MAX_ITERATIONS = 20
#NUM_EPOCHS
#BATCH_SIZE
NN_type = 'LSTM'
UNITS_NUMBER = 128
USER_NUMBER = len(dsTrain)

for i in range(0,MAX_ITERATIONS):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}, metrics={}'.format(i,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 
    
    
    current_time2 = datetime.datetime.now()
    time_stamp2 = current_time2.timestamp()
    
    print('')
    print('awake')
    res,resA = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res,resA = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    print(i,"End timestamp:", time_stamp2,current_time2)
    print('')
    print('')

0 Start timestamp: 1678104794.106594 2023-03-06 12:13:14.106594


2023-03-06 12:13:23.901097: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


round  0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8447548), ('precision', 0.84339243), ('loss', 0.37151137), ('num_examples', 1222125), ('num_batches', 38199)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])



2023-03-06 12:29:29.236288: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


4259/4259 [==============================] - 13s 3ms/step
0.9861501 0.013849913 0.5
[[0.92592406 0.07407597]
 [0.9281725  0.07182752]
 [0.9248759  0.07512413]
 ...
 [0.8429487  0.1570513 ]
 [0.843146   0.15685397]
 [0.8433021  0.15669796]]

awake
Accuracy: 0.701377
Precision: 0.701377
Recall: 1.000000
F1 score: 0.824481
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[    0 40697]
 [    0 95585]]

asleep
Accuracy: 0.703072
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix


/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[95816     0]
 [40466     0]]

Global
2
accuracy:  0.7022240648067977
precision:  0.3506882787161914
recall:  0.5
f1_score:  0.41224063795192933
cohen_kappa_score:  0.0
roc_auc_score:  0.5

0 End timestamp: 1678104794.106594 2023-03-06 12:29:42.287443


1 Start timestamp: 1678105782.759004 2023-03-06 12:29:42.759004
round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.85472023), ('precision', 0.8533767), ('loss', 0.350944), ('num_examples', 1222125), ('num_batches', 38199)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 10s 2ms/step
0.9943 0.00569997 0.5
[[0.94183224 0.05816774]
 [0.944608   0.05539207]
 [0.9407778  0.05922215]
 ...
 [0.8742536  0.12574647]
 [0.8744534  0.12554654]
 [0.8746033  0.12539671]]

awake
Accuracy: 0.701369
Precision: 0.701386
Recall: 0.999948
F1 scor

4259/4259 [==============================] - 10s 2ms/step
0.99318475 0.0068152477 0.5
[[0.9076396  0.09236031]
 [0.9131689  0.08683112]
 [0.9066233  0.09337676]
 ...
 [0.84803075 0.15196924]
 [0.8482286  0.15177144]
 [0.848388   0.15161194]]

awake
Accuracy: 0.710974
Precision: 0.709640
Recall: 0.995062
F1 score: 0.828456
Cohens kappa: 0.053240
ROC AUC: 0.519400
\Confusion Matrix
[[ 1780 38917]
 [  472 95113]]

asleep
Accuracy: 0.712625
Precision: 0.789076
Recall: 0.043913
F1 score: 0.083197
Cohens kappa: 0.053567
ROC AUC: 0.519478
\Confusion Matrix
[[95341   475]
 [38689  1777]]

Global
2
accuracy:  0.7117997974787573
precision:  0.7493580047360888
recall:  0.5194876977505103
f1_score:  0.45582655397399674
cohen_kappa_score:  0.053403511572818474
roc_auc_score:  0.5194389611929686

7 End timestamp: 1678111564.997733 2023-03-06 14:22:20.577108


8 Start timestamp: 1678112541.028775 2023-03-06 14:22:21.028775
round  8, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDic

Cohens kappa: 0.086012
ROC AUC: 0.532037
\Confusion Matrix
[[94312  1504]
 [37238  3228]]

Global
2
accuracy:  0.7149403442861126
precision:  0.6987026714769895
recall:  0.5320650711516115
f1_score:  0.48566612304265994
cohen_kappa_score:  0.08581808423097498
roc_auc_score:  0.5319931056390308

13 End timestamp: 1678117461.694996 2023-03-06 16:00:29.075038


14 Start timestamp: 1678118429.573015 2023-03-06 16:00:29.573015
round  14, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8465599), ('precision', 0.8452122), ('loss', 0.36115834), ('num_examples', 1222125), ('num_batches', 38199)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 11s 2ms/step
0.9917442 0.008255814 0.5
[[0.89341295 0.1065871 ]
 [0.9002623  0.09973773]
 [0.89365935 0.10634066]
 ...
 [0.8228982  0.17710173]
 [0.8231

In [56]:
roundData = []

columns = ['NN_type','units','epochs','batch_size','max_iterations','Users',
           'round_iteration','start_time','end_time','round_time_s','round_time_m',
           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
           'TP','FP','FN','TN']

MAX_ITERATIONS = 2
#NUM_EPOCHS
#BATCH_SIZE
NN_type = 'LSTM'
UNITS_NUMBER = 128
USER_NUMBER = len(dsTrain)

generalData = [NN_type,UNITS_NUMBER,NUM_EPOCHS,BATCH_SIZE,MAX_ITERATIONS,USER_NUMBER]

for i in range(0,MAX_ITERATIONS):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}, metrics={}'.format(i,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 
    
    # generate time metrics
    current_time2 = datetime.datetime.now()
    time_stamp2 = current_time2.timestamp()
    processing_time_s = (time_stamp2-time_stamp)
    # generate general metrics
    rowData = [i,current_time,current_time2,processing_time_s,(processing_time_s)/60]

    print('')
    print('awake')    
    res,resA = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
   
    #columns = ['NN_type','units','epochs','batch_size','max_iterations',''Users',
    #            round_iteration','start_time','end_time','round_time_s','round_time_m',
    #           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
    #           'TP','FP','FN','TN']
    # new data
    classData = np.concatenate((['awake'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    roundData.append(classData)
    
    print('')
    print('asleep')
    res,resA = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    # new data
    classData = np.concatenate((['asleep'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    roundData.append(classData)
    print('')
    print('Global')
    resA = showGlobalMetrics(test) #return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score
    # new data
    classData = np.concatenate((['avg'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    roundData.append(classData)
    print('')
    print(i,"End timestamp:", time_stamp2,current_time2)
    print('')
    print('')

0 Start timestamp: 1678128424.083334 2023-03-06 18:47:04.083334
round  0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.84883213), ('precision', 0.847487), ('loss', 0.3576298), ('num_examples', 1222125), ('num_batches', 38199)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 11s 3ms/step
0.990173 0.009827017 0.5
[[0.8798914  0.12010869]
 [0.8875469  0.1124531 ]
 [0.880972   0.11902791]
 ...
 [0.8007983  0.19920167]
 [0.8010397  0.1989603 ]
 [0.80124444 0.19875559]]

awake
Accuracy: 0.722318
Precision: 0.731053
Recall: 0.955673
F1 score: 0.828407
Cohens kappa: 0.163864
ROC AUC: 0.564956
\Confusion Matrix
[[ 7091 33606]
 [ 4237 91348]]

asleep
Accuracy: 0.723705
Precision: 0.624117
Recall: 0.174715
F1 score: 0.273005


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Cohens kappa: 0.164485
ROC AUC: 0.565138
\Confusion Matrix
[[91558  4258]
 [33396  7070]]

Global
2
accuracy:  0.7230118430900632
precision:  0.6775851295762716
recall:  0.5651937682188103
f1_score:  0.5507056810919613
cohen_kappa_score:  0.164174296924529
roc_auc_score:  0.565046770438148

0 End timestamp: 1678129575.341398 2023-03-06 19:06:15.341398


1 Start timestamp: 1678129575.863745 2023-03-06 19:06:15.863745


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8490277), ('precision', 0.84769315), ('loss', 0.35714474), ('num_examples', 1222125), ('num_batches', 38199)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 11s 3ms/step
0.9898605 0.010139538 0.5
[[0.8791466  0.12085345]
 [0.88679415 0.11320586]
 [0.88030374 0.11969624]
 ...
 [0.7978192  0.20218073]
 [0.7980762  0.20192379]
 [0.7982705  0.20172952]]

awake
Accuracy: 0.723280
Precision: 0.732577
Recall: 0.953549
F1 score: 0.828583
Cohens kappa: 0.170721
ROC AUC: 0.567998
\Confusion Matrix
[[ 7425 33272]
 [ 4440 91145]]

asleep
Accuracy: 0.724622
Precision: 0.623767
Recall: 0.182894
F1 score: 0.282853
Cohens kappa: 0.171269
ROC AUC: 0.568152
\Confusion Matrix


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


[[91352  4464]
 [33065  7401]]

Global
2
accuracy:  0.7239510720417957
precision:  0.678172060482505
recall:  0.5682217393344416
f1_score:  0.5557183760714858
cohen_kappa_score:  0.17099505936324028
roc_auc_score:  0.5680750157624155

1 End timestamp: 1678130589.542687 2023-03-06 19:23:09.542687




<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [57]:
dataMetrics = pd.DataFrame(data=roundData,columns=columns) 

dataMetrics

,NN_type,units,epochs,batch_size,max_iterations,Users,round_iteration,start_time,end_time,round_time_s,...,precision,recall,f1_score,cohen_kappa_score,roc_auc_score,confusion_matrix,TP,FP,FN,TN
0,LSTM,128,3,32,2,20,0,2023-03-06 18:47:04.083334,2023-03-06 19:06:15.341398,1151.258064,...,0.731053,0.955673,0.828407,0.163864,0.564956,"[[7091, 33606], [4237, 91348]]",7091.0,33606.0,4237.0,91348.0
1,LSTM,128,3,32,2,20,0,2023-03-06 18:47:04.083334,2023-03-06 19:06:15.341398,1151.258064,...,0.624117,0.174715,0.273005,0.164485,0.565138,"[[91558, 4258], [33396, 7070]]",91558.0,4258.0,33396.0,7070.0
2,LSTM,128,3,32,2,20,0,2023-03-06 18:47:04.083334,2023-03-06 19:06:15.341398,1151.258064,...,0.6775851295762716,0.5651937682188103,0.5507056810919613,0.164174296924529,0.565046770438148,None,NaN,NaN,NaN,NaN
3,LSTM,128,3,32,2,20,1,2023-03-06 19:06:15.863745,2023-03-06 19:23:09.542687,1013.678942,...,0.732577,0.953549,0.828583,0.170721,0.567998,"[[7425, 33272], [4440, 91145]]",7425.0,33272.0,4440.0,91145.0
4,LSTM,128,3,32,2,20,1,2023-03-06 19:06:15.863745,2023-03-06 19:23:09.542687,1013.678942,...,0.623767,0.182894,0.282853,0.171269,0.568152,"[[91352, 4464], [33065, 7401]]",91352.0,4464.0,33065.0,7401.0
5,LSTM,128,3,32,2,20,1,2023-03-06 19:06:15.863745,2023-03-06 19:23:09.542687,1013.678942,...,0.678172060482505,0.5682217393344416,0.5557183760714858,0.17099505936324028,0.5680750157624155,None,NaN,NaN,NaN,NaN


In [55]:
roundData = []
current_time2 = datetime.datetime.now()
time_stamp2 = current_time2.timestamp()
processing_time_s = (time_stamp2-time_stamp)
generalData = [NN_type,UNITS_NUMBER,NUM_EPOCHS,BATCH_SIZE,MAX_ITERATIONS,USER_NUMBER]
rowData = [i,current_time,current_time2,processing_time_s,(processing_time_s)/60]

print('')
print('awake')    
res,resA = printMetrics(y_test_label_label['awake'],y_test2['awake'])
test.append(res)

#columns = ['NN_type','units','epochs','batch_size','max_iterations',''Users',
#            round_iteration','start_time','end_time','round_time_s','round_time_m',
#           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
#           'TP','FP','FN','TN']
# new data
classData = np.concatenate((['awake'], resA))
classData = np.concatenate((rowData, classData))
classData = np.concatenate((generalData, classData))
roundData.append(classData)

print('')
print('asleep')
res,resA = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
test.append(res)
# new data
classData = np.concatenate((['asleep'], resA))
classData = np.concatenate((rowData, classData))
classData = np.concatenate((generalData, classData))
roundData.append(classData)
print('')
print('Global')
resA = showGlobalMetrics(test) #return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score
# new data
classData = np.concatenate((['avg'], resA))
classData = np.concatenate((rowData, classData))
classData = np.concatenate((generalData, classData))
roundData.append(classData)
print('')
print(i,"End timestamp:", time_stamp2,current_time2)
print('')
print('')


awake
Accuracy: 0.720902
Precision: 0.728930
Recall: 0.958519
F1 score: 0.828106
Cohens kappa: 0.154052
ROC AUC: 0.560666
\Confusion Matrix
[[ 6626 34071]
 [ 3965 91620]]

asleep
Accuracy: 0.722304
Precision: 0.623737
Recall: 0.163248


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


F1 score: 0.258770
Cohens kappa: 0.154632
ROC AUC: 0.560829
\Confusion Matrix
[[91831  3985]
 [33860  6606]]

Global
20
accuracy:  0.7216029996624647
precision:  0.6763338038262073
recall:  0.5608833774811359
f1_score:  0.5434378430527997
cohen_kappa_score:  0.15434162169948298
roc_auc_score:  0.5607474019089275

0 End timestamp: 1678128404.236884 2023-03-06 18:46:44.236884




<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [58]:
dataMetrics = pd.DataFrame(data=roundData,columns=columns) 

dataMetrics

,NN_type,units,epochs,batch_size,max_iterations,Users,round_iteration,start_time,end_time,round_time_s,...,precision,recall,f1_score,cohen_kappa_score,roc_auc_score,confusion_matrix,TP,FP,FN,TN
0,LSTM,128,3,32,2,20,0,2023-03-06 18:47:04.083334,2023-03-06 19:06:15.341398,1151.258064,...,0.731053,0.955673,0.828407,0.163864,0.564956,"[[7091, 33606], [4237, 91348]]",7091.0,33606.0,4237.0,91348.0
1,LSTM,128,3,32,2,20,0,2023-03-06 18:47:04.083334,2023-03-06 19:06:15.341398,1151.258064,...,0.624117,0.174715,0.273005,0.164485,0.565138,"[[91558, 4258], [33396, 7070]]",91558.0,4258.0,33396.0,7070.0
2,LSTM,128,3,32,2,20,0,2023-03-06 18:47:04.083334,2023-03-06 19:06:15.341398,1151.258064,...,0.6775851295762716,0.5651937682188103,0.5507056810919613,0.164174296924529,0.565046770438148,None,NaN,NaN,NaN,NaN
3,LSTM,128,3,32,2,20,1,2023-03-06 19:06:15.863745,2023-03-06 19:23:09.542687,1013.678942,...,0.732577,0.953549,0.828583,0.170721,0.567998,"[[7425, 33272], [4440, 91145]]",7425.0,33272.0,4440.0,91145.0
4,LSTM,128,3,32,2,20,1,2023-03-06 19:06:15.863745,2023-03-06 19:23:09.542687,1013.678942,...,0.623767,0.182894,0.282853,0.171269,0.568152,"[[91352, 4464], [33065, 7401]]",91352.0,4464.0,33065.0,7401.0
5,LSTM,128,3,32,2,20,1,2023-03-06 19:06:15.863745,2023-03-06 19:23:09.542687,1013.678942,...,0.678172060482505,0.5682217393344416,0.5557183760714858,0.17099505936324028,0.5680750157624155,None,NaN,NaN,NaN,NaN


In [59]:
dataMetrics['accuracy']

0              0.722318
1              0.723705
2    0.7230118430900632
3               0.72328
4              0.724622
5    0.7239510720417957
Name: accuracy, dtype: object

In [52]:
(time_stamp2)
time_stamp

1678125968.614807

In [53]:
current_time2 = datetime.datetime.now()

In [54]:
dataMetrics = pd.DataFrame(data=roundData,columns=columns) 

dataMetrics

,NN_type,units,epochs,batch_size,max_iterations,Users,round_iteration,start_time,end_time,round_time_s,...,precision,recall,f1_score,cohen_kappa_score,roc_auc_score,confusion_matrix,TP,FP,FN,TN
0,LSTM,128,3,32,1,20,0,2023-03-06 18:06:08.614807,2023-03-06 18:39:35.870053,2007.255246,...,0.72893,0.958519,0.828106,0.154052,0.560666,"[[6626, 34071], [3965, 91620]]",6626.0,34071.0,3965.0,91620.0
1,LSTM,128,3,32,1,20,0,2023-03-06 18:06:08.614807,2023-03-06 18:39:35.870053,2007.255246,...,0.623737,0.163248,0.25877,0.154632,0.560829,"[[91831, 3985], [33860, 6606]]",91831.0,3985.0,33860.0,6606.0
2,LSTM,128,3,32,1,20,0,2023-03-06 18:06:08.614807,2023-03-06 18:39:35.870053,2007.255246,...,0.6763338038262074,0.560883377481136,0.5434378430527996,0.15434162169948296,0.5607474019089275,None,NaN,NaN,NaN,NaN


In [18]:
result = fed_avg_process.next(state, federated_training_data[0:10])
state = result.state
metrics = result.metrics
print('round  1, metrics={}'.format(metrics))

2023-03-04 09:20:21.718658: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.82992435), ('precision', 0.8286222), ('loss', 0.39952663), ('num_examples', 572898), ('num_batches', 17907)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [19]:
#def keras_evaluate(state, round_num):
    # Take our global model weights and push them back into a Keras model to
    # use its standard `.evaluate()` method.
keras_model = create_keras_model()


keras_model.compile(
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision()])

# get neural network weights
weights = fed_avg_process.get_model_weights(state)
weights.assign_weights_to(keras_model)


#tttt.model.assign_weights_to(keras_model)
yhat_probs = keras_model.predict(X_test_data)

maxX = yhat_probs.max()
minX = yhat_probs.min()
avgX = yhat_probs.mean()

print(maxX,minX,avgX)
print(yhat_probs)
# predict crisp classes for test set deprecated
#printMetrics(y_test,yhat_probs)
test = list()

xx = yhat_probs.round()

y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

print('')
print('awake')
res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

2023-03-04 09:28:16.713330: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


4259/4259 [==============================] - 13s 2ms/step
0.97919613 0.02080385 0.5
[[0.9219694  0.07803061]
 [0.9234769  0.07652311]
 [0.92058057 0.07941942]
 ...
 [0.7984561  0.2015439 ]
 [0.79863083 0.2013692 ]
 [0.79873574 0.20126428]]

awake
Accuracy: 0.701377
Precision: 0.701377
Recall: 1.000000
F1 score: 0.824481
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[    0 40697]
 [    0 95585]]

asleep
Accuracy: 0.703072
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[95816     0]
 [40466     0]]

Global
2
accuracy:  0.7022240648067977
precision:  0.3506882787161914
recall:  0.5
f1_score:  0.41224063795192933
cohen_kappa_score:  0.0
roc_auc_score:  0.5


/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

print('')
print('awake')
res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)


awake
Accuracy: 0.701377
Precision: 0.701377
Recall: 1.000000
F1 score: 0.824481
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[    0 40697]
 [    0 95585]]

asleep
Accuracy: 0.703072
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[95816     0]
 [40466     0]]

Global
4
accuracy:  0.7022240648067977
precision:  0.3506882787161914
recall:  0.5
f1_score:  0.41224063795192933
cohen_kappa_score:  0.0
roc_auc_score:  0.5


/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
result = fed_avg_process.next(state, federated_training_data[10:19])
state = result.state
metrics = result.metrics
print('round  2, metrics={}'.format(metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8549116), ('precision', 0.85364234), ('loss', 0.34650168), ('num_examples', 615294), ('num_batches', 19231)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [22]:
#def keras_evaluate(state, round_num):
    # Take our global model weights and push them back into a Keras model to
    # use its standard `.evaluate()` method.
keras_model = create_keras_model()


keras_model.compile(
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.CategoricalAccuracy()])

# get neural network weights
weights = fed_avg_process.get_model_weights(state)
weights.assign_weights_to(keras_model)


#tttt.model.assign_weights_to(keras_model)
yhat_probs = keras_model.predict(X_test_data)

maxX = yhat_probs.max()
minX = yhat_probs.min()
avgX = yhat_probs.mean()

print(maxX,minX,avgX)
print(yhat_probs)
# predict crisp classes for test set deprecated
#printMetrics(y_test,yhat_probs)
test = list()

xx = yhat_probs.round()

y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

print('')
print('awake')
res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

4259/4259 [==============================] - 10s 2ms/step
0.9965713 0.0034286503 0.5
[[0.9485654  0.05143452]
 [0.95140064 0.04859938]
 [0.94741374 0.05258626]
 ...
 [0.8934536  0.10654639]
 [0.89368105 0.10631891]
 [0.89386046 0.10613954]]

awake
Accuracy: 0.701355
Precision: 0.701370
Recall: 0.999969
F1 score: 0.824466
Cohens kappa: -0.000044
ROC AUC: 0.499984
\Confusion Matrix
[[    0 40697]
 [    3 95582]]

asleep
Accuracy: 0.703050
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
Cohens kappa: -0.000044
ROC AUC: 0.499984
\Confusion Matrix
[[95813     3]
 [40466     0]]

Global
2
accuracy:  0.7022020516282415
precision:  0.3506849918182552
recall:  0.49998430716116543
f1_score:  0.41223303315736803
cohen_kappa_score:  -4.402504078360181e-05
roc_auc_score:  0.4999843260778692


In [23]:
for i in range(0,2):
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))
    
    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    print('')
    print('')

round  2-4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8500882), ('precision', 0.8488199), ('loss', 0.35842997), ('num_examples', 1188192), ('num_batches', 37138)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 11s 3ms/step
0.9948612 0.0051388214 0.5
[[0.93161756 0.06838237]
 [0.93516773 0.06483224]
 [0.93000305 0.06999692]
 ...
 [0.8724655  0.1275345 ]
 [0.8726728  0.12732725]
 [0.8728424  0.12715757]]

awake
Accuracy: 0.701274
Precision: 0.701423
Recall: 0.999582
F1 score: 0.824371
Cohens kappa: 0.000309
ROC AUC: 0.500110
\Confusion Matrix
[[   26 40671]
 [   40 95545]]

asleep
Accuracy: 0.702969
Precision: 0.393939
Recall: 0.000643
F1 score: 0.001283
Cohens kappa: 0.000316
ROC AUC: 0.500113
\Confusion Matrix
[[95776    40]
 [40440    26]]

Global
2
accuracy:  0.7021213366402

In [24]:
for i in range(2,4):
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    print('')
    print('')

round  6-8, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8445445), ('precision', 0.843261), ('loss', 0.36485735), ('num_examples', 1188192), ('num_batches', 37138)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 11s 2ms/step
0.9941597 0.0058402787 0.5
[[0.91629636 0.08370368]
 [0.9209168  0.07908326]
 [0.91455835 0.08544163]
 ...
 [0.86035436 0.1396457 ]
 [0.860551   0.139449  ]
 [0.86071634 0.13928361]]

awake
Accuracy: 0.708076
Precision: 0.706733
Recall: 0.997855
F1 score: 0.827434
Cohens kappa: 0.035068
ROC AUC: 0.512663
\Confusion Matrix
[[ 1118 39579]
 [  205 95380]]

asleep
Accuracy: 0.709771
Precision: 0.845049
Recall: 0.027628
F1 score: 0.053507
Cohens kappa: 0.035371
ROC AUC: 0.512744
\Confusion Matrix
[[95611   205]
 [39348  1118]]

Global
2
accuracy:  0.70892340881407

In [25]:
import datetime;
  
current_time = datetime.datetime.now()
  
time_stamp = current_time.timestamp()
print("Start timestamp:-", time_stamp,current_time)

Start timestamp:- 1677926580.303852 2023-03-04 10:43:00.303852


In [26]:
for i in range(4,6):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

10 Start timestamp: 1677926580.318815 2023-03-04 10:43:00.318815
round  10-12, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.842666), ('precision', 0.8413883), ('loss', 0.36770174), ('num_examples', 1188192), ('num_batches', 37138)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 9s 2ms/step
0.9934964 0.006503508 0.5
[[0.9046663  0.09533372]
 [0.9101642  0.08983584]
 [0.9031202  0.09687976]
 ...
 [0.8467884  0.15321152]
 [0.8469849  0.15301505]
 [0.84716284 0.15283716]]

awake
Accuracy: 0.711789
Precision: 0.711107
Recall: 0.992143
F1 score: 0.828440
Cohens kappa: 0.061983
ROC AUC: 0.522732
\Confusion Matrix
[[ 2170 38527]
 [  751 94834]]

asleep
Accuracy: 0.713410
Precision: 0.741185
Recall: 0.053502
F1 score: 0.099799
Cohens kappa: 0.062310
ROC AUC: 0.522806
\Confusion Matrix
[[9

In [27]:
for i in range(6,8):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

14 Start timestamp: 1677928505.205672 2023-03-04 11:15:05.205672
round  14-16, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8426719), ('precision', 0.8413893), ('loss', 0.3680452), ('num_examples', 1188192), ('num_batches', 37138)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 10s 2ms/step
0.99284416 0.0071557993 0.5
[[0.8965413  0.10345873]
 [0.9026778  0.0973222 ]
 [0.89534444 0.1046556 ]
 ...
 [0.83464915 0.16535087]
 [0.8348593  0.16514067]
 [0.8350304  0.16496964]]

awake
Accuracy: 0.713476
Precision: 0.714115
Recall: 0.986358
F1 score: 0.828443
Cohens kappa: 0.079307
ROC AUC: 0.529459
\Confusion Matrix
[[ 2953 37744]
 [ 1304 94281]]

asleep
Accuracy: 0.715069
Precision: 0.692037
Recall: 0.072802
F1 score: 0.131744
Cohens kappa: 0.079724
ROC AUC: 0.529560
\Confusion Matrix


In [28]:
for i in range(7,8):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

16 Start timestamp: 1677930417.669901 2023-03-04 11:46:57.669901
round  16-18, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8431222), ('precision', 0.84183955), ('loss', 0.36730176), ('num_examples', 1188192), ('num_batches', 37138)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 10s 2ms/step
0.9925753 0.0074247387 0.5
[[0.89209735 0.10790269]
 [0.89869976 0.10130022]
 [0.8913366  0.10866334]
 ...
 [0.8293319  0.17066817]
 [0.8295586  0.17044137]
 [0.8297454  0.17025462]]

awake
Accuracy: 0.714115
Precision: 0.716310
Recall: 0.980855
F1 score: 0.827965
Cohens kappa: 0.091188
ROC AUC: 0.534239
\Confusion Matrix
[[ 3566 37131]
 [ 1830 93755]]

asleep
Accuracy: 0.715648
Precision: 0.658821
Recall: 0.087852
F1 score: 0.155030
Cohens kappa: 0.091556
ROC AUC: 0.534319
\Confusion Matrix

In [ ]:
for i in range(8,16):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

18 Start timestamp: 1677931366.945084 2023-03-04 12:02:46.945084
round  18-20, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8431819), ('precision', 0.8418951), ('loss', 0.36699367), ('num_examples', 1188192), ('num_batches', 37138)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 10s 2ms/step
0.99236065 0.0076394184 0.5
[[0.88960344 0.11039658]
 [0.89640325 0.10359678]
 [0.88902324 0.11097678]
 ...
 [0.82586944 0.17413051]
 [0.8260937  0.1739063 ]
 [0.82628566 0.17371437]]

awake
Accuracy: 0.714614
Precision: 0.717491
Recall: 0.978312
F1 score: 0.827844
Cohens kappa: 0.097472
ROC AUC: 0.536789
\Confusion Matrix
[[ 3877 36820]
 [ 2073 93512]]

asleep
Accuracy: 0.716118
Precision: 0.649412
Recall: 0.095488
F1 score: 0.166494
Cohens kappa: 0.097815
ROC AUC: 0.536858
\Confusion Matrix